# What do we want?

1. To create a CSV of questions
    - Each question will have a subject
    - There will be a folder called `images` to hold any pictures related to the question
2. We want to be able to keep track of some data points about each quiz
3. We want to have an email sent to us each day with a link to a page
    - On the page, we'll answer the questions
    - Once done, the quiz results will be saved and persisted

# Step 1: Creating questions

- This will be easy
    - Will do it in Excel and save to folder

In [1]:
import pandas as pd

In [15]:
df_test_questions = pd.read_csv('test_quiz.csv')

In [16]:
df_test_questions

,Category,Question,Options,Correct Answer,Image Filename
0,Math,What is 1 + 1?,"1, 2, 3, 4, 5",2,1.png
1,Math,What is 2+2?,"1, 2, 3, 4, 5",4,2.png
2,Geography,What is the capital of Ontario?,"Toronto, Ottawa, Vancouver",Toronto,NaN


# Step 2: Writing function to read in questions from CSV and create question widgets

- First, we define our `create_multipleChoice_widget` function

In [8]:
from ipywidgets import widgets, Layout, Box

In [9]:
def create_multipleChoice_widget(description, options, correct_answer, img_filepath = None):
    if correct_answer not in options:
        options.append(correct_answer)
    
    correct_answer_index = options.index(correct_answer)
    
    alphabet = 'abcdefghijklmnopqrstuvwxyz'
    options_w_letters = [f'{alphabet[i]}) {option}' for i, option in enumerate(options)]
    
    radio_options = [(words, i) for i, words in enumerate(options_w_letters)]
    
    alternativ = widgets.RadioButtons(
        options = radio_options,
        description = '',
        disabled = False,
        indent = False,
        align = 'center',
    )
    
    description_out = widgets.Output(layout=Layout(width='auto'))
    
    if img_filepath:
        image = open(img_filepath, 'rb').read()
        image_out = widgets.Image(value=image, 
                                  format='png', 
                                  width='500px', 
                                  height='auto',
                                 )
    
    with description_out:
        print(description)
        
    feedback_out = widgets.Output()

    def check_selection(b):
        a = int(alternativ.value)
        if a==correct_answer_index:
            s = '\x1b[6;30;42m' + "correct" + '\x1b[0m' +"\n"
        else:
            correct_answer_letter = alphabet[correct_answer_index]
            s = '\x1b[5;30;41m' + f"incorrect, the correct answer was {correct_answer_letter}" + '\x1b[0m' +"\n"
        with feedback_out:
            print(s)
            global quiz_results
            tuple_results = (s, a==correct_answer_index)
            quiz_results.append(tuple_results)
            alternativ.close()
            check.layout.display = 'none'
        return
    
    check = widgets.Button(description="check")
    check.on_click(check_selection)
    
    check_out = widgets.HBox([check])
    
    if img_filepath:
            return widgets.VBox([description_out, 
                         image_out,
                         alternativ, 
                         widgets.HBox([check]), feedback_out], 
                        layout=Layout(display='flex',
                                     flex_flow='column',
                                     align_items='stretch',
                                     width='auto')) 
    
    return widgets.VBox([description_out, 
                         alternativ, 
                         check_out, 
                         feedback_out], 
                        layout=Layout(
                            display='flex',
                            flex_flow='column',
                            align_items='stretch',
                            width='auto')) 

- Next, we define a function whose parameter is the CSV of questions

In [24]:
def create_questions(df_questions):
    list_widgets = []
    for question_idx, s_question in df_questions.iterrows():
        description = s_question.loc['Question']
        options = s_question.loc['Options'].split(',')
        correct_answer = s_question.loc['Correct Answer']
        image_filename = s_question.loc['Image Filename']
        
        if '.png' in str(image_filename):
            image_filepath = f'images/{image_filename}'
        else:
            image_filepath = None
        
        question_widget = create_multipleChoice_widget(description, options, correct_answer)
        list_widgets.append(question_widget)
    return list_widgets

In [26]:
list_widgets = create_questions(df_test_questions)